In [1]:
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import re
import numpy as np
import scipy
import copy
import re
from keras.layers import  CuDNNLSTM

Using TensorFlow backend.


In [3]:
#master_data = pd.read_csv("multilabels.csv")
#master_data = pd.read_csv("train.csv")
master_data = pd.read_csv("multiclass.csv")

In [4]:
features = np.array(master_data.iloc[:, 1])

In [5]:
labels = np.array(master_data.iloc[:, 0])

In [6]:
test = features
p_data = []

for eachPoint in test:
    p_data.append([re.sub(r"(http|ftp|https)\S+", "", str(eachPoint))])

In [7]:
test = p_data

In [8]:
def lowercase(arr):
    return np.char.lower(arr)

In [9]:
def removeStopWords(arr):
    stopWords = set(stopwords.words('english'))
    
    filteredWords = []
    for a in arr:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            if w not in stopWords:
                wordsFiltered = wordsFiltered + " " + w
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [10]:
def removeSymbols(arr):
    symbols = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    for i in range(len(symbols)):
        arr = np.char.replace(arr, symbols[i], '')   
    return np.char.replace(arr, "  ", " ")

In [11]:
def stemming(word):
    porter_stemmer=PorterStemmer()
    filteredWords = []
    for a in word:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            wordsFiltered = wordsFiltered + " " + porter_stemmer.stem(w)
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [12]:
def removeImplicitNoise(arr):
    return np.char.replace (arr, 'NEWLINE_TOKEN', '')

In [13]:
def processClean(data):
    a = removeImplicitNoise(data)
    a = lowercase(a)
    a = removeStopWords(a)
    a = removeSymbols(a)
    a = stemming(a)
    return a

a = processClean(test)

In [14]:
list_l = []

for i in a:
    list_l.append(str(i[0]))

In [15]:
def make_grams(data, n = 2):
    
    grammed_data = []
    
    for i in data:

        k = copy.deepcopy(i)

        for r in range(2,n+1):
            sixgrams = ngrams(i.split(), r)

            for grams in sixgrams:
                g = ""
                for p in grams:
                    g = g+p

                k = k+" "+g
        grammed_data.append(k)
    
    return grammed_data

grammed_data = make_grams(list_l)

In [16]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(feature_vector.shape)

(4500, 81065)


In [17]:
# TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(vector.shape)

(4500, 81065)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.20, random_state=42)

In [19]:
from keras.utils import to_categorical

data = y_train
print(data)

def encode(data):
    encoded = to_categorical(data,dtype='int')
    return encoded

encoded_data = encode(data)
print(encoded_data)

[2 2 1 ... 2 2 0]
[[0 0 1]
 [0 0 1]
 [0 1 0]
 ...
 [0 0 1]
 [0 0 1]
 [1 0 0]]


In [20]:
data = y_test
print(data)

def encode(data):
    encoded = to_categorical(data,dtype='int')
    return encoded

encoded_data_test = encode(data)
print(encoded_data_test)

[1 2 1 1 0 1 1 0 2 2 0 1 2 0 0 2 0 2 1 1 1 1 2 2 2 0 1 0 2 0 2 2 0 2 2 2 1
 1 2 2 0 0 1 2 0 1 1 2 0 0 2 2 1 2 2 0 0 0 1 0 2 2 0 1 0 1 0 0 2 0 2 2 1 0
 1 0 2 0 1 2 1 0 1 1 1 0 1 1 0 2 0 2 2 0 1 0 2 1 1 1 1 1 2 1 2 1 1 2 0 2 2
 1 0 1 1 2 2 2 2 1 0 1 1 1 1 2 2 1 2 1 0 0 0 2 1 2 0 1 0 2 2 1 2 0 0 1 2 1
 0 2 2 0 0 2 0 1 1 0 0 1 0 1 0 2 1 2 2 0 0 2 2 1 0 2 0 0 2 0 0 2 2 2 2 0 1
 1 0 0 0 1 0 0 2 0 2 2 2 1 2 0 1 1 0 2 1 1 2 2 2 1 0 0 2 0 1 0 1 0 1 1 2 2
 2 2 1 0 2 2 0 0 2 2 1 0 2 2 2 2 1 0 1 1 2 2 2 0 2 2 1 1 0 1 2 1 0 2 2 2 2
 0 1 0 0 0 1 1 0 2 2 1 1 0 0 2 2 0 0 2 1 2 0 2 0 2 1 2 1 1 0 1 0 2 0 1 2 2
 1 2 1 1 0 0 0 2 0 1 2 0 0 0 2 0 1 2 0 2 1 0 2 2 1 0 1 0 1 2 1 2 0 2 0 2 1
 0 0 1 1 1 0 2 0 1 0 0 0 2 2 0 2 1 0 2 2 1 1 1 1 1 2 0 2 2 0 0 0 2 1 0 2 0
 0 1 1 1 1 0 1 2 1 0 2 1 2 2 1 2 0 1 2 0 2 0 2 0 0 2 0 0 0 2 0 2 1 2 0 0 1
 1 2 0 1 0 2 1 0 1 1 0 1 0 2 2 1 2 1 1 0 2 0 0 0 0 1 2 1 2 2 2 0 2 2 2 0 0
 1 0 0 2 2 1 0 2 0 0 0 0 1 2 2 2 0 0 1 1 1 0 0 1 1 1 0 0 2 0 1 0 0 2 1 2 1
 1 2 1 0 2 0 0 0 1 0 1 1 

In [21]:
# LSTM and CNN for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import LSTM, CuDNNLSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


In [22]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re

import sys
import os

# os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model


In [23]:

MAX_SEQUENCE_LENGTH = 1000
MAX_SEQUENCE_LENGTH = 81065
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
word_index=81065

In [24]:

# embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
embedding_matrix = np.random.random((word_index + 1, EMBEDDING_DIM))

In [63]:

MAX_SEQUENCE_LENGTH = 81065
convs = []
filter_sizes = [3,4,5]

       
embedding_layer = Embedding(81065, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
# l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
# l_pool2 = MaxPooling1D(5)(l_cov2)
# l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
# l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool1)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(3, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 81065)             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 81065, 100)        8106500   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 81061, 128)        64128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 16212, 128)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2075136)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               265617536 
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total para

In [25]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split

sequence_length = 81065
vocabulary_size = 81065 # 18765
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 5
batch_size = 30

In [26]:
encoded_data[0].reshape(1,3)[0]

array([0, 0, 1])

In [27]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D

# from __future__ import print_function

import numpy as np

from keras.layers import Dense, Input, Flatten, Reshape, concatenate, Dropout
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras import optimizers
from keras import regularizers

In [28]:
sequence_length = 81065
embedding_layer = Embedding(sequence_length,
                                EMBEDDING_DIM,
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)


print('Training model.')

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)


# add first conv filter
embedded_sequences = Reshape((MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, 1))(embedded_sequences)
x = Conv2D(100, (5, EMBEDDING_DIM), activation='relu')(embedded_sequences)
x = MaxPooling2D((MAX_SEQUENCE_LENGTH - 5 + 1, 1))(x)


# add second conv filter.
y = Conv2D(100, (4, EMBEDDING_DIM), activation='relu')(embedded_sequences)
y = MaxPooling2D((MAX_SEQUENCE_LENGTH - 4 + 1, 1))(y)


# add third conv filter.
z = Conv2D(100, (3, EMBEDDING_DIM), activation='relu')(embedded_sequences)
z = MaxPooling2D((MAX_SEQUENCE_LENGTH - 3 + 1, 1))(z)


# concate the conv layers
alpha = concatenate([x,y,z])

# flatted the pooled features.
alpha = Flatten()(alpha)

# dropout
alpha = Dropout(0.5)(alpha)

# predictions
preds = Dense(3, activation='softmax')(alpha)

# build model
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta()

model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

Training model.
(?, 81065, 100)


In [29]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 81065)        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 81065, 100)   8106500     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 81065, 100, 1 0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 81061, 1, 100 50100       reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [30]:
model.fit(X_train, encoded_data, batch_size=50, epochs=2)

Epoch 1/2
 500/3600 [===>..........................] - ETA: 2:00 - loss: 1.0960 - acc: 0.3620

KeyboardInterrupt: 

In [ ]:
from keras.utils import to_categorical
def encode(data):
    encoded = to_categorical(data,dtype='int')
    return encoded

text_labels=encode(y_test)
y_predicted=[]
for i in range(len(X_test)):
    prediction = model.predict(np.array([X_test[i]]))
    predicted_label = np.argmax(prediction[0])
    y_predicted.append(predicted_label)

print(f1_score(y_test,y_predicted,average=None))
print(classification_report(y_test,y_predicted))